In [7]:
from Distillation_disc_cont import simulator
from Actor_Critic import Agent
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
import dill

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#dill.load_session('Agent.db')

In [9]:
%autoreload

[autoreload of Distillation_disc_cont failed: Traceback (most recent call last):
  File "C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\IPython\extensions\autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\IPython\extensions\autoreload.py", line 387, in update_generic
    update(a, b)
  File "C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\IPython\extensions\autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\IPython\extensions\autoreload.py", line 280, in update_instances
    frame = next(frame_nfo.frame for frame_nfo in inspect.stack()
StopIteration
]


In [10]:
env = simulator()
agent = Agent(env = env, alpha = 1e-6, beta = 1e-7, gamma = 0.99)
agent.actor.summary()
#agent.critic.summary()
#agent.policy.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 6)            0                                            
__________________________________________________________________________________________________
fc1 (Dense)                     (None, 20)           140         Input[0][0]                      
__________________________________________________________________________________________________
fc2 (Dense)                     (None, 20)           420         fc1[0][0]                        
__________________________________________________________________________________________________
fc3 (Dense)                     (None, 20)           420         fc2[0][0]                        
____________________________________________________________________________________________

In [ ]:
score_history = []
num_episodes = int(1e4)
for i in range(num_episodes):
	done = False
	
	state = env.reset()
	score = 0
	
	while not done:
		action = agent.choose_action_epsgreedy(state, i, num_episodes)
		next_state, reward, done, info = env.step(action)
		agent.learn(state,  action, reward, next_state, done)
		state = next_state
		score += reward
	
	score_history.append(score)
	avg_score = np.mean(score_history[-100:]) #average of last 100 scores
	if i%100 == 0:
		print(f'episode {i}, avg_score {avg_score}, last action {action}')
        


D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:64: RuntimeWarning: invalid value encountered in double_scalars
  LK_B = bots[Light_Key]/sum(bots)


episode 0, avg_score -20044.326273422193, last action (3, array([[0.9457034]]))


D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:74: RuntimeWarning: invalid value encountered in log
  N =  np.log(LK_D/(1-LK_D) * (1-LK_B)/LK_B)/np.log(self.relative_volatility[Light_Key])
D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:63: RuntimeWarning: invalid value encountered in double_scalars
  LK_D = tops[Light_Key]/sum(tops)


episode 100, avg_score -17631.8131412606, last action (3, array([[0.30667746]]))
episode 200, avg_score -16639.251991015106, last action (3, array([[0.999]], dtype=float32))
episode 300, avg_score -13189.882601523861, last action (3, array([[0.66011442]]))
episode 400, avg_score -12095.150431569893, last action (1, array([[0.999]], dtype=float32))
episode 500, avg_score -11094.858020070535, last action (3, array([[0.999]], dtype=float32))
episode 600, avg_score -10894.834000597675, last action (2, array([[0.999]], dtype=float32))
episode 700, avg_score -8999.264342402566, last action (1, array([[0.999]], dtype=float32))
episode 800, avg_score -9316.786259701927, last action (2, array([[0.39506849]]))
episode 900, avg_score -10274.003607900097, last action (2, array([[0.999]], dtype=float32))
episode 1000, avg_score -11386.329135626023, last action (1, array([[0.999]], dtype=float32))
episode 1100, avg_score -10416.644419876842, last action (4, array([[0.999]], dtype=float32))
episode 1

episode 9100, avg_score -8146.48697937499, last action (0, array([[0.999]], dtype=float32))
episode 9200, avg_score -8989.199088643834, last action (1, array([[0.999]], dtype=float32))
episode 9300, avg_score -9136.309702816705, last action (3, array([[0.999]], dtype=float32))
episode 9400, avg_score -9320.990669509132, last action (0, array([[0.999]], dtype=float32))
episode 9500, avg_score -8942.731234772986, last action (4, array([[0.99854482]]))
episode 9600, avg_score -8190.726942785722, last action (1, array([[0.999]], dtype=float32))
episode 9700, avg_score -10085.90729916713, last action (3, array([[0.999]], dtype=float32))
episode 9800, avg_score -9136.81748560058, last action (3, array([[0.999]], dtype=float32))


In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N

In [ ]:
episodes = np.arange(num_episodes)
smoothed_rews = running_mean(score_history, 10)
plt.plot(episodes[-len(smoothed_rews):], smoothed_rews)
plt.plot(episodes, score_history,color='grey', alpha=0.3)
plt.xlabel("steps")
plt.ylabel("reward")
plt.legend(["avg reward", "reward"])

In [ ]:
#test agent
state = env.reset()
done = False
while done == False:
    state = state[np.newaxis, :]
    action_discrete, action_continuous = agent.policy.predict(state)
    action_discrete = np.argmax(action_discrete)
    action = action_discrete, action_continuous
    
    state, reward, done, info = env.step(action)
    print(f'action LK: {action_discrete}, action split: {action_continuous[0]}, reward: {reward}')

In [ ]:
env.stream_table

In [10]:
action_discrete, action_continuous = agent.policy.predict(state[np.newaxis, :])

In [ ]:
#dill.dump_session('Agent.db')